# 📝 License and Notices

This notebook is part of "Arduino 2D Pan-Tilt Control System + AI Mosquito Auto-Tracking" (mosquito-pt2d).

Copyright 2025 Arduino PT2D Project

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this work except in compliance with the License. You may obtain a copy of the License at:

- http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

For project and third-party notices, see the repository files:
- LICENSE
- NOTICE

## 1. 掛載 Google Drive

## 📋 完整工作流程總覽

### 階段 1: Colab 訓練 (本 Notebook)

| 步驟 | Cell | 輸出 | 說明 |
|------|------|------|------|
| 1. 掛載 Drive | Cell 2 | - | Google Drive 連結 |
| 2. 安裝環境 | Cell 3 | - | ultralytics, torch |
| 3. 準備數據集 | Cell 4 | `training_dataset/` | 80/20 分割 |
| 4. 訓練模型 | Cell 5 | `runs/detect/mosquito_dual_platform/` | 100 epochs |
| 5. 評估結果 | Cell 6 | mAP50, mAP50-95 | 精度指標 |
| 6. 保存模型 | Cell 7 | `models/mosquito_yolov8_new.pt` | ⚠️ 關鍵檔案 |
| 7. 路徑驗證 | Cell 8 | - | 確認無斷點 |
| 8. 查看曲線 | Cell 9 | 訓練圖表 | 可視化 |

### 階段 2: 本機部署 (deploy_model.py)

```bash
# 前置條件: Google Drive 已同步
cd d:/Workspaces/mosquito-pt2d
python python/deploy_model.py
```

| 步驟 | 動作 | 輸出 | 說明 |
|------|------|------|------|
| 1. 備份 | 自動 | `mosquito_yolov8_backup_*.pt` | 保護舊模型 |
| 2. 複製 | 自動 | `models/mosquito_yolov8.pt` | 從 Google Drive |
| 3. ONNX | 自動 | `models/mosquito_yolov8.onnx` | 兩平台共用 |
| 4. RKNN | 自動 | `models/mosquito_yolov8.rknn` | Orange Pi 5 |
| 5. RDK X5 | 手動 | `mosquito_yolov8.bin` | 需 hb_mapper |

### 階段 3: 平台部署

**Orange Pi 5** (自動完成):
```bash
# 直接使用 .rknn 模型
python python/streaming_tracking_system.py
```

**RDK X5** (需額外步驟):
```bash
# 1. x86 機器編譯 .bin
python3 mapper.py --onnx models/mosquito_yolov8.onnx --cal-images calibration_images

# 2. 複製到板端並更新 config.py
scp mosquito_yolov8.bin rdkx5:/path/to/models/

# 3. 運行追蹤系統
python python/streaming_tracking_system.py
```

### 🔗 關鍵斷點檢查

✅ **Colab → Google Drive**
- 輸出路徑: `/content/drive/MyDrive/Colab Notebooks/mosquito-training/models/mosquito_yolov8_new.pt`

✅ **Google Drive → 本機**
- 自動同步到: `G:\我的雲端硬碟\Colab Notebooks\mosquito-training\models\`
- deploy_model.py 自動推導路徑（基於 config.RELOCATION_BASE_DIR）

✅ **本機 → 目標板**
- Orange Pi 5: 直接使用 `models/mosquito_yolov8.rknn`
- RDK X5: 手動複製 `mosquito_yolov8.bin`

### 🚀 一鍵命令

```bash
# 完整流程（Orange Pi 5）
python python/deploy_model.py && python python/streaming_tracking_system.py

# 僅更新模型
python python/deploy_model.py

# 跳過 RKNN（僅 ONNX）
python python/deploy_model.py --skip-rknn
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 設定工作目錄（與本機搬遷/部署預期一致）
import os
os.makedirs('/content/drive/MyDrive/Colab Notebooks/mosquito-training', exist_ok=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/mosquito-training')

print("✓ Google Drive 已掛載")
print(f"✓ 工作目錄: {os.getcwd()}")

## 2. 安裝訓練環境

In [ ]:
!pip install ultralytics -q

# 檢查 GPU
import torch
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 名稱: {torch.cuda.get_device_name(0)}")
    print(f"GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ 警告：未偵測到 GPU，請檢查執行階段設定")

## 3. 準備訓練數據集

In [ ]:
import os
import shutil
from pathlib import Path
import random

# 建立目錄結構
os.makedirs('training_dataset/images/train', exist_ok=True)
os.makedirs('training_dataset/images/val', exist_ok=True)
os.makedirs('training_dataset/labels/train', exist_ok=True)
os.makedirs('training_dataset/labels/val', exist_ok=True)

# 選擇樣本來源：優先使用 relocated/<latest>/mosquito，其次 fallback 到 sample_collection/confirmed/mosquito
relocated_base = Path('relocated')
mosquito_dir: Path
if relocated_base.exists():
    subdirs = [d for d in relocated_base.iterdir() if d.is_dir()]
    if subdirs:
        latest = sorted(subdirs)[-1]  # 目錄名稱為時間戳，字串排序即可
        mosquito_dir = latest / 'mosquito'
        print(f"✓ 使用搬遷資料夾樣本: {mosquito_dir}")
    else:
        mosquito_dir = Path('sample_collection/confirmed/mosquito')
        print(f"⚠️ 找不到 relocated/* 子目錄，改用: {mosquito_dir}")
else:
    mosquito_dir = Path('sample_collection/confirmed/mosquito')
    print(f"⚠️ 找不到 relocated 目錄，改用: {mosquito_dir}")

mosquito_images = list(mosquito_dir.glob('*.jpg'))

if len(mosquito_images) == 0:
    print("❌ 錯誤：找不到蚊子樣本圖片")
    print(f"請確認 {mosquito_dir} 目錄存在且包含圖片")
else:
    # 80/20 分割
    random.shuffle(mosquito_images)
    split_idx = int(len(mosquito_images) * 0.8)
    train_images = mosquito_images[:split_idx]
    val_images = mosquito_images[split_idx:]

    # 複製圖片並生成標籤
    for img_list, split in [(train_images, 'train'), (val_images, 'val')]:
        for img_path in img_list:
            # 複製圖片
            shutil.copy(img_path, f'training_dataset/images/{split}/')

            # 生成 YOLO 標籤（整張圖為蚊子）
            label_path = f'training_dataset/labels/{split}/{img_path.stem}.txt'
            with open(label_path, 'w') as f:
                f.write('0 0.5 0.5 1.0 1.0\n')  # class_id x_center y_center width height

    print(f"✓ 訓練集: {len(train_images)} 張")
    print(f"✓ 驗證集: {len(val_images)} 張")

    # 建立 dataset.yaml（使用當前工作目錄的絕對路徑）
    yaml_content = """path: /content/drive/MyDrive/Colab Notebooks/mosquito-training/training_dataset
train: images/train
val: images/val
names:
  0: mosquito
nc: 1
"""

    with open('training_dataset/dataset.yaml', 'w') as f:
        f.write(yaml_content)

    print("✓ 數據集準備完成")

## 4. 開始訓練模型

In [ ]:
from ultralytics import YOLO

# 載入預訓練模型
# 建議：使用 YOLOv8n（nano）以獲得最佳性能/精度平衡
model = YOLO('yolov8n.pt')  # 或使用已訓練的 'models/mosquito_yolov8.pt'

# 訓練參數（適配 RDK X5 與 Orange Pi 5）
# RDK X5 參考: https://github.com/D-Robotics/rdk_model_zoo
# Orange Pi 5 參考: https://github.com/airockchip/rknn-toolkit2
results = model.train(
    data='training_dataset/dataset.yaml',
    epochs=100,            # 充足訓練以提升量化後精度
    imgsz=640,             # 兩平台標準輸入尺寸（YOLOv8n: 640×640）
    batch=16,              # 根據 GPU 記憶體調整
    lr0=0.01,              # 學習率（YOLOv8 預設）
    name='mosquito_dual_platform',
    patience=20,           # 增加 patience 避免過早停止
    device=0,              # 使用 GPU
    workers=4,             # 資料載入執行緒
    amp=True,              # 自動混合精度訓練（加速）
    # 量化友好設定
    close_mosaic=10,       # 最後 10 個 epoch 關閉 Mosaic 增強
    optimizer='AdamW',     # 優化器
    cos_lr=True            # 餘弦學習率衰減
)

print("\n" + "="*50)
print("✓ 訓練完成!")
print(f"✓ 最佳模型: runs/detect/mosquito_dual_platform/weights/best.pt")
print("="*50)

## 5. 評估訓練結果

In [ ]:
# 驗證模型
metrics = model.val()

print("\n訓練指標：")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"精確率: {metrics.box.mp:.4f}")
print(f"召回率: {metrics.box.mr:.4f}")

## 7. (可選) 查看訓練曲線

In [ ]:
from IPython.display import Image, display

# 顯示訓練結果圖表
print("訓練曲線：")
display(Image('runs/detect/mosquito_dual_platform/results.png'))

print("\n混淆矩陣：")
display(Image('runs/detect/mosquito_dual_platform/confusion_matrix.png'))

## 8. 模型格式轉換（全平台一鍵生成）

將訓練好的 PyTorch 模型轉換為所有部署格式：
- **ONNX**: 通用格式
- **RKNN**: Orange Pi 5 (RK3588 NPU) 專用格式
- **BIN**: RDK X5 (Bayes-e BPU) 專用格式

完成後，`deploy_model.py` 只需複製模型文件即可。

In [ ]:
# 安裝模型轉換工具
print("📦 安裝模型轉換工具...")

import platform
is_x86 = platform.machine() == 'x86_64'

if not is_x86:
    print("⚠️ 警告: 需要 x86_64 架構，當前架構:", platform.machine())
    print("   僅能生成 ONNX 模型")
else:
    # 1. 安裝 RDK X5 工具鏈（hb_mapper）
    print("安裝 hb_mapper (RDK X5)...")
    !pip install horizon-tc-ui -i https://pypi.tuna.tsinghua.edu.cn/simple --quiet
    !pip install onnx==1.12.0 onnxsim==0.4.33 --quiet

    # 2. 安裝 rknn-toolkit2（Orange Pi 5）
    print("安裝 rknn-toolkit2 (Orange Pi 5)...")
    !pip install rknn-toolkit2 -i https://pypi.tuna.tsinghua.edu.cn/simple --quiet

    print("✅ 工具鏈安裝完成")

    # 驗證安裝
    !hb_mapper --version

    # 驗證 rknn-toolkit2
    try:
        from rknn.api import RKNN
        print("✅ rknn-toolkit2 可用")
    except ImportError:
        print("⚠️ rknn-toolkit2 導入失敗")

In [ ]:
# 準備校準數據集（用於 BIN 模型量化）
import shutil
import random
from pathlib import Path

calib_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/calibration_images')
calib_dir.mkdir(parents=True, exist_ok=True)

# 從訓練集隨機抽取 50 張作為校準數據
train_images = list(Path('/content/training_dataset/train/images').glob('*.jpg'))
random.seed(42)
calib_samples = random.sample(train_images, min(50, len(train_images)))

print(f"📸 準備 {len(calib_samples)} 張校準圖片...")
for img in calib_samples:
    shutil.copy(img, calib_dir / img.name)

print(f"✅ 校準數據集準備完成: {calib_dir}")
print(f"   圖片數量: {len(list(calib_dir.glob('*.jpg')))}")

In [ ]:
# 導出 ONNX 模型（通用格式）
from ultralytics import YOLO
from pathlib import Path
import shutil
from datetime import datetime

# 設定路徑
model_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/models')
model_dir.mkdir(parents=True, exist_ok=True)

# 使用訓練輸出的最佳模型
best_pt = Path('runs/detect/mosquito_dual_platform/weights/best.pt')

if not best_pt.exists():
    print("❌ 錯誤：找不到訓練輸出的 best.pt")
    print(f"   預期路徑: {best_pt}")
else:
    # 1. 先保存 PyTorch 模型到 Google Drive
    print("📦 保存 PyTorch 模型...")
    pt_save_path = model_dir / 'mosquito_yolov8_new.pt'
    shutil.copy(best_pt, pt_save_path)
    print(f"✅ PyTorch 模型已保存: {pt_save_path}")

    # 同時備份一份帶日期的版本
    date_str = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_path = model_dir / f'mosquito_yolov8_backup_{date_str}.pt'
    shutil.copy(best_pt, backup_path)
    print(f"✅ 備份版本已保存: {backup_path}")

    # 2. 導出 ONNX
    print("\n📦 導出 ONNX 模型...")
    model = YOLO(str(best_pt))
    model.export(format='onnx', imgsz=640, opset=12, simplify=True)

    # 將導出的 ONNX 移動到 models 目錄
    onnx_path = Path('runs/detect/mosquito_dual_platform/weights/best.onnx')
    if onnx_path.exists():
        onnx_save_path = model_dir / 'mosquito_yolov8.onnx'
        shutil.copy(onnx_path, onnx_save_path)
        print(f"✅ ONNX 模型已保存: {onnx_save_path}")
    else:
        print("⚠️ ONNX 導出失敗，請檢查錯誤信息")

In [ ]:
# 創建 hb_mapper 配置文件
from pathlib import Path

model_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/models')
calib_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/calibration_images')
onnx_model = model_dir / 'mosquito_yolov8.onnx'
config_path = Path('/content/yolov8_config.yaml')

config_content = f"""model_parameters:
  onnx_model: '{str(onnx_model)}'
  march: 'bayes-e'
  layer_out_dump: False
  working_dir: '/content/model_output'
  output_model_file_prefix: 'mosquito_yolov8'

input_parameters:
  input_name: 'images'
  input_type_rt: 'nv12'
  input_layout_rt: 'NHWC'
  input_type_train: 'rgb'
  input_layout_train: 'NCHW'
  norm_type: 'data_scale'
  scale_value: 0.003921568627451
  input_shape: '1x3x640x640'

calibration_parameters:
  cal_data_dir: '{str(calib_dir)}'
  cal_data_type: 'float32'
  calibration_type: 'default'

compiler_parameters:
  compile_mode: 'latency'
  debug: False
  optimize_level: 'O3'
"""

config_path.write_text(config_content)
print(f"✅ 配置文件已創建: {config_path}")

In [ ]:
# 執行 hb_mapper 編譯（RDK X5 BIN 格式）
import platform
from pathlib import Path
import shutil

model_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/models')

if platform.machine() == 'x86_64':
    print("🔧 開始編譯 RDK X5 BIN 模型...")
    print("⏱️  預計耗時: 5-10 分鐘\n")

    !hb_mapper makertbin --config /content/yolov8_config.yaml

    # 檢查輸出
    bin_model = Path('/content/model_output/mosquito_yolov8.bin')
    if bin_model.exists():
        print("\n✅ BIN 模型編譯成功!")
        print(f"   輸出路徑: {bin_model}")
        print(f"   文件大小: {bin_model.stat().st_size / 1024 / 1024:.2f} MB")

        # 複製到 models 目錄
        output_bin = model_dir / 'mosquito_yolov8.bin'
        shutil.copy(bin_model, output_bin)
        print(f"   已複製到: {output_bin}")
    else:
        print("\n❌ BIN 模型編譯失敗，請檢查上方錯誤信息")
else:
    print("⚠️ 跳過 BIN 編譯（需要 x86_64 架構）")

In [ ]:
# 生成 RKNN 模型（Orange Pi 5）
import platform
from pathlib import Path

model_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/models')
onnx_model = model_dir / 'mosquito_yolov8.onnx'
calib_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/calibration_images')

if platform.machine() == 'x86_64':
    print("🔧 開始生成 Orange Pi 5 RKNN 模型...")
    print("⏱️  預計耗時: 3-5 分鐘\n")

    try:
        from rknn.api import RKNN

        # 初始化 RKNN 物件
        rknn = RKNN(verbose=False)

        # 配置模型
        print("📝 配置 RKNN 參數...")
        rknn.config(
            mean_values=[[0, 0, 0]],
            std_values=[[255, 255, 255]],
            target_platform='rk3588'
        )

        # 載入 ONNX 模型
        print("📦 載入 ONNX 模型...")
        ret = rknn.load_onnx(model=str(onnx_model))
        if ret != 0:
            print('❌ 載入 ONNX 模型失敗!')
        else:
            # 使用校準數據集進行量化
            print("⚙️  執行量化（使用校準數據集）...")
            ret = rknn.build(
                do_quantization=True,
                dataset=str(calib_dir)
            )

            if ret != 0:
                print('❌ RKNN build 失敗!')
            else:
                # 導出 RKNN 模型
                rknn_file = model_dir / 'mosquito_yolov8.rknn'
                ret = rknn.export_rknn(str(rknn_file))

                if ret != 0:
                    print('❌ 導出 RKNN 模型失敗!')
                else:
                    print(f"\n✅ RKNN 模型生成成功!")
                    print(f"   輸出路徑: {rknn_file}")
                    print(f"   文件大小: {rknn_file.stat().st_size / 1024 / 1024:.2f} MB")

        # 釋放資源
        rknn.release()

    except ImportError:
        print("⚠️ rknn-toolkit2 未安裝，跳過 RKNN 生成")
    except Exception as e:
        print(f"❌ RKNN 生成失敗: {e}")
else:
    print("⚠️ 跳過 RKNN 生成（需要 x86_64 架構）")

In [ ]:
# 模型轉換完成摘要
import zipfile
from datetime import datetime
from pathlib import Path

model_dir = Path('/content/drive/MyDrive/Colab Notebooks/mosquito-training/models')

print("\n" + "="*50)
print("📊 模型轉換完成")
print("="*50)

# 檢查生成的模型
pt_file = model_dir / 'mosquito_yolov8_new.pt'
onnx_file = model_dir / 'mosquito_yolov8.onnx'
rknn_file = model_dir / 'mosquito_yolov8.rknn'
bin_file = model_dir / 'mosquito_yolov8.bin'

print("\n已生成的模型:")
model_files = []
if pt_file.exists():
    print(f"  ✅ PyTorch: {pt_file.name} ({pt_file.stat().st_size / 1024 / 1024:.2f} MB)")
    model_files.append(pt_file)
if onnx_file.exists():
    print(f"  ✅ ONNX: {onnx_file.name} ({onnx_file.stat().st_size / 1024 / 1024:.2f} MB)")
    model_files.append(onnx_file)
if rknn_file.exists():
    print(f"  ✅ RKNN (Orange Pi 5): {rknn_file.name} ({rknn_file.stat().st_size / 1024 / 1024:.2f} MB)")
    model_files.append(rknn_file)
if bin_file.exists():
    print(f"  ✅ BIN (RDK X5): {bin_file.name} ({bin_file.stat().st_size / 1024 / 1024:.2f} MB)")
    model_files.append(bin_file)

# 創建帶日期戳記的 ZIP 備份
if model_files:
    print("\n" + "="*50)
    print("📦 創建備份壓縮檔")
    print("="*50)

    date_str = datetime.now().strftime('%Y%m%d_%H%M%S')
    zip_filename = model_dir / f'mosquito_models_backup_{date_str}.zip'

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for model_file in model_files:
            zipf.write(model_file, model_file.name)
            print(f"  ✅ 已加入: {model_file.name}")

    zip_size_mb = zip_filename.stat().st_size / 1024 / 1024
    print(f"\n✅ 備份完成: {zip_filename.name} ({zip_size_mb:.2f} MB)")
    print(f"   位置: {zip_filename}")

print("\n📥 下一步:")
print("  1. 等待 Google Drive 同步完成")
print("  2. 執行 python/deploy_model.py（自動複製所有模型到專案目錄）")
print("  3. 系統會自動選擇對應平台的模型格式:")
print("     - Orange Pi 5 → mosquito_yolov8.rknn")
print("     - RDK X5 → mosquito_yolov8.bin")